# Case 1 (Part I): House price prediction

In this case (Part I), you will build a multilayer perceptron network to predict the selling price of properties. The dataset consists of all single family houses and condos that were sold in Denver in a given year.

You need to submit the following files on canvas site:

- A report in the pdf format containing the plots of the training errors for the multi-layer perception model and the linear regression model, and the answers to the two questions below. You should also provide interpretations and implications of each plot/table in your report. It is not enough to simply put a chart or a table of numbers in the report and expect the audience to understand what the chart means and what it implies. The point is to provide some insights for an audience like senior management at Zillow.

- The complete Jupyter notebook containing all your Pytorch code with explanations, along with a Markdown text explaining different parts if needed.




---
## Kaggle community competition: Prof. X's Prize


You need to set up a Kaggle account and joined the Kaggle competition by following the [link](https://www.kaggle.com/t/414a77c12150407d97e39fae245e34ef).

- Name your team as Section_X_Team_Y, where X is either A or B or C or D, and Y is your team number.
- One of the team members can serve as team leader and invite other members of your team to join the team.

- Each team can submit at most 20 predictions daily

To get the test error for your model, you need to submit your predicted prices for test data on Kaggle. See Kaggle competition website for more detailed instructions. Note that in Part I of the case, you do not need to worry about optimizing your model to get the lowest error possible. The Part I will be graded based on your implemention of the base models as specified below.  We will come back to optimize the model and compete for Prof. X's Prize in Part II of the case.

---
## Data Loading and Visualize Data


The train data and test data are available on the Kaggle competition website.
You need to first download them, then upload them to the google colab, and then read the data using pandas.

In [ ]:
import pandas as pd  # Importing pandas, which is a library for data manipulation and analysis
#Read the datasets
train_df =pd.read_csv("train.csv")
test_df =pd.read_csv("test.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

### Visualization of SALE PRICES in train data

Let's take a closer look at the sale prices in the train data.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt  # Importing matplotlib's pyplot for making plots and charts

# Set the style
sns.set(style="whitegrid")

# Create a histogram
plt.figure(figsize=(10, 6))
sns.histplot(train_df['SALE_PRICE'], bins=50, color='blue')
plt.title('Histogram of Sale Prices (Train Data)')
plt.xlabel('Sale Price')
plt.ylabel('Number of Properties')
plt.show()

NameError: name 'train_df' is not defined

<Figure size 1000x600 with 0 Axes>

---
## Data Preparation

The first step when building a neural network model is getting your data into the proper form to feed into the network.

- **Train labels**: We need to extract the sale prices from the train data as train labels. Since the house prices can take very large values, to make training fast it is helpful to define the train labels as the sale prices divided by a normalization factor.

- **Handing non-numeric features**: Some of the house features are non-numeric. We will learn about how to process categorical data in the upcoming lectures. For now, you can  remove those non-numeric features and only train over the numeric features.

- **Feature standardization**: When predicting house prices, you started from features that took a variety of ranges—some features had small floating-point values, and others had fairly large integer values. The model might be able to automatically adapt to such heterogeneous data, but it would definitely make learning more difficult. A widespread best practice for dealing with such data is to do feature-wise normalization: for each feature in the input data (a column in the input dataframe), we subtract the mean of the feature and divide by the standard deviation, so that the feature is centered around 0 and has
a unit standard deviation. **Note**: We need to ensure that the train and test data go through the same normalization.

- **Handling missing values**: There may exist some entries with missing values. After the feature standardization, we can impute the missing values with zeros.

In [ ]:
import pandas as pd
import numpy as np

#Train labels
train_label = train_df['SALE_PRICE']
train_df = train_df.drop('SALE_PRICE', axis=1)


# Handling non-numeric features
numeric_cols_train = train_df.select_dtypes(include=np.number).columns
numeric_cols_test = test_df.select_dtypes(include=np.number).columns
train_df_numeric = train_df[numeric_cols_train]
test_df_numeric = test_df[numeric_cols_test]


# Feature standardization
def standardize_features(df, mean_values, std_values):
    df_standardized = (df - mean_values) / std_values
    return df_standardized

mean_values_train = train_df_numeric.mean()
std_values_train = train_df_numeric.std()

mean_values_test = test_df_numeric.mean()
std_values_test = test_df_numeric.std()

train_df_standardized = standardize_features(train_df_numeric, mean_values_train, std_values_train)
test_df_standardized = standardize_features(test_df_numeric, mean_values_test, std_values_test)


# Handling missing values
train_df_final = train_df_standardized.fillna(0)
test_df_final = test_df_standardized.fillna(0)


We see that the sale_price in train data has a wide range from 50K to 2 million, with the median price 431K. We can divide the sale_price by 100K, so the normalized sale_price is between 0.5 and 20 in training data. Remember, when we output the predicted price for the test data, we need to multiply back the normalization factor.

In [ ]:
#TODO: define labels for train data as the sale prices divided by $100,000
normalization_factor = 100000
train_labels = train_label / normalization_factor

In [ ]:
train_labels.shape

In [ ]:
#TODO: Write code to construct feature vectors for train and test data after data preparation.
train_features = train_df_final
test_features = test_df_final

In [ ]:
train_features.shape, test_features.shape

Finally, we convert features and labels to PyTorch tensors.

In [ ]:
import torch
import numpy as np

# Convert training features and labels to PyTorch tensors
train_features = torch.tensor(train_features.values.astype(np.float32), dtype=torch.float32)
test_features = torch.tensor(test_features.values.astype(np.float32), dtype=torch.float32)
train_labels = torch.tensor(train_labels.values.reshape(-1, 1).astype(np.float32), dtype=torch.float32)

---
## DataLoaders and Batching

After creating training, test, and validation data, we can create DataLoaders for this data by following two steps:
1. Create a known format for accessing our data, using [TensorDataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset) which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets. Note that we will shuffle the train data, so the model will not learn a particular order. For test data, we do not shuffle.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
#  Create DataLoaders and batch our train data
train_data = TensorDataset(train_features, train_labels)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
#TODO: Create DataLoaders and batch for test data
test_data = TensorDataset(test_features)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

Let's take a batch to have a sanity check

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
features, labels = next(dataiter)

print('Sample input size: ', features.size()) # batch_size, seq_length
print('Sample input: \n', features)
print()
print('Sample label size: ', labels.size()) # batch_size
print('Sample label: \n', labels)

---
## Linear regression as benchmark

Let us first build a linear regression model as a benchmark.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

#TODO: Build a linear regression model network
lin_net = nn.Linear(train_features.shape[1], 1)

Let's take a batch and see the output

In [ ]:
features, labels = next(dataiter)
output=lin_net(features)
output.shape,labels.shape

## Train the model

First, we will use GPU training if it is availabe.

In [ ]:
#TODO: use GPU for training if it is availabe
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('Training on GPU')
else:
  device = torch.device('cpu')
  print('Training on CPU')

lin_net = lin_net.to(device)

Second, let us specify the loss function.

In [ ]:
#TODO: specify the loss function for training
criterion = nn.MSELoss()

We are now ready to train the network.


Note that with house prices, as with stock prices, we care about relative quantities more than absolute quantities. Thus we tend to care more about the relative error than about the absolute error. For instance, if our prediction is off by \\$100,000 when estimating the sale price of a house which is \\$125,000, then we are probably doing a horrible job. On the other hand, if we err by this amount for a house with sale price \\$2 million, this might represent a pretty  accurate prediction.

To this end, we will use the median error rate (MER) used by [Zestimate](https://www.zillow.com/z/zestimate/) to measure the predictive performance. The error rate is defined as
$$
\text{Error Rate} = \left| \frac{\text{Predicted Price}-\text{Actual Price}}{\text{Actual Price}} \right|
$$
The median error rate is defined as the median of error rates for all properties.

In [ ]:
#TODO: Write code to train the network
optimizer = optim.SGD(lin_net.parameters(), lr=0.01)

epochs = 100
train_losses = []

for epoch in range(epochs):
  running_loss = 0.0
  for features, labels in train_loader:
      features, labels = features.to(device), labels.to(device)

      optimizer.zero_grad()

      outputs = lin_net(features)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()

  train_losses.append(running_loss / len(train_loader))
  print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}')

Plot the training error (MER) over epochs

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn

optimizer = optim.SGD(lin_net.parameters(), lr=0.01)

epochs = 100
train_mers = []

for epoch in range(epochs):
    epoch_errors = []

    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = lin_net(features)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        with torch.no_grad():
            errors = torch.abs((outputs - labels) / labels) * 100
            epoch_errors.extend(errors.cpu().numpy())

    epoch_mer = torch.median(torch.tensor(epoch_errors)).item()
    train_mers.append(epoch_mer)
    print(f'Epoch {epoch+1}, MER: {epoch_mer:.4f}%')


import matplotlib.pyplot as plt

plt.plot(range(1, epochs + 1), train_mers, label='Train MER')
plt.xlabel('Epochs')
plt.ylabel('Median Error Rate (%)')
plt.legend()
plt.title('Training MER over Epochs')
plt.show()


---
## Build the Multi-layer Perceptron Base Model

In the following, we build a multi-layer perception model.

In [ ]:
#TODO: Build a multi-layer perception neural network with 2 hidden layers of sizes 256 and 128, respectively and ReLu activations
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

input_size = train_features.shape[1]
hidden_size1 = 256
hidden_size2 = 128
output_size = 1

mlp_net = MLP(input_size, hidden_size1, hidden_size2, output_size).to(device)

In [ ]:
#TODO: write code to train the MLP network
optimizer = optim.Adam(mlp_net.parameters(), lr=0.001)

# Using 100 epochs
epochs = 100
mlp_train_losses = []

for epoch in range(epochs):
  running_loss = 0.0
  for features, labels in train_loader:
    features, labels = features.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = mlp_net(features)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  mlp_train_losses.append(running_loss / len(train_loader))
  print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}')

In [ ]:
#TODO: Write code to plot the training error (MER) over epochs
# Plot
import torch
import torch.optim as optim
import torch.nn as nn

optimizer = optim.SGD(mlp_net.parameters(), lr=0.01)

epochs = 100
train_mers = []

for epoch in range(epochs):
    epoch_errors = []

    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        # Zero out the parameter gradients
        optimizer.zero_grad()

        # Forward
        outputs = mlp_net(features)

        # Backpropagation loss
        loss = criterion(outputs, labels)

        # Backward + optimize
        loss.backward()
        optimizer.step()

        # Error percentage
        with torch.no_grad():
            errors = torch.abs((outputs - labels) / labels) * 100
            epoch_errors.extend(errors.cpu().numpy())

    # MER
    epoch_mer = torch.median(torch.tensor(epoch_errors)).item()
    train_mers.append(epoch_mer)
    print(f'Epoch {epoch+1}, MER: {epoch_mer:.4f}%')


# Plot
import matplotlib.pyplot as plt

plt.plot(range(1, epochs + 1), train_mers, label='Train MER')
plt.xlabel('Epochs')
plt.ylabel('Median Error Rate (%)')
plt.legend()
plt.title('Training MER over Epochs')
plt.show()

**Question 1**: What are your final training errors of the multilayer perception model and the linear regression model?

At epoch 100, the median error rate for the linear model is 22.0745% and the median error rate for the multilayer perceptron model is 13.0823%.

---
## Inference on test data

After the MLP model is trained, we can use it for inference.

In [ ]:
#TODO: write the code to generate predicted sale prices for test data. using mlp model

predicted_prices = []
mlp_net.eval()

with torch.no_grad():
    for data in test_loader:
        inputs = data[0].to(device)
        outputs = mlp_net(inputs)
        predicted_prices.extend(outputs.cpu().numpy())

predicted_prices = np.array(predicted_prices).flatten() * normalization_factor

In [ ]:
#TODO: save the predicted sale prices into submission_csv
import pandas as pd
submission_df = pd.read_csv('sample_submission.csv')

In [ ]:
submission_df['SALE_PRICE'] = predicted_prices

In [ ]:
submission_df.to_csv('submission.csv', index=False)

Now, we can submit our predictions on Kaggle and see how they compare with the actual house prices (labels) on the test set.

- Log in to the Kaggle website and visit the house price prediction competition page.

- Click the “Submit Predictions”.

- Click the “Browse Files” button in the dashed box at the bottom of the page and select the prediction file you wish to upload.

- Click the “Submit” button at the bottom of the page to view your results.

**Question 2**: What is the test error shown on Kaggle? How does it compare with the train error?

The training error in Kaggle is 0.14566. It is a bit higher than the training error we got which was 13.0823%.